In [3]:
%load_ext autoreload
%autoreload 2
import yaml
import pandas as pd
from pandarallel import pandarallel
import click
import numpy as np
from tss.manuscript.utils import expand_anno_id
from tss.manuscript.peak_distance import create_anno_tss

genome="GCF"
thresh=0
numSamp=1
with open("manuscript_params.yaml") as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    params=yaml.load(file)
params

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/isshamie/software/anaconda2/envs/TSS/lib/python3.7/site-packages/ipykernel_launcher.py:17: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.


{'OUTDIR': '/data/isshamie/TSS/Analysis/TSS_code/data/processed/manuscript',
 'genome_fasta': '/data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna',
 'annotation': {'synapse': '/data/isshamie/genome/hamster/syn20999279_picr.gff',
  'GCF': '/data/isshamie/genome/hamster/ncbi_anno_103/GCF_003668045.1_CriGri-PICR/GCF_003668045.1_CriGri-PICR_genomic.gff'},
 'ncrna_keys': {'synapse': ['lncRNA'],
  'GCF': ['snoRNA', 'snRNA', 'miRNA', 'lnc_RNA']},
 'MERGE_TISSUES': ['all', 'bmdm'],
 'atac_f': '/data/isshamie/TSS/Analysis/Analysis_NCBI_PICR/Results/ATAC_results/CHO/ATAC_ppr.naive_overlap.narrowPeak.sort',
 'all_atac': '/data/isshamie/TSS/Analysis/Analysis_NCBI_PICR/Results/ATAC_results/ATAC_merge.bed',
 'META_SAMPLES': '/data/isshamie/TSS/Analysis/Analysis_03302020/parameters/samples.tsv',
 'peaks_dir': '/data/isshamie/TSS/Analysis/Analysis_NCBI_PICR/f04_peaks/',
 'cpm_thresh': [1, 0],
 'numsamples_thresh': [2, 1]}

### init anno files and parameters

In [5]:
cpu=32
gff_f= "/data/isshamie/genome/hamster/ncbi_anno_103/GCF_003668045.1_CriGri-PICR/GCF_003668045.1_CriGri-PICR_genomic.gff"
reftss_f=f"data/{genome}/genome/refTSS.tsv"
cds_f = f"data/{genome}/genome/CDS.bed"
intron_f = f"data/{genome}/genome/introns.bed"
pc= "mRNA"#.split(',')
ncrna= params['ncrna_keys'][genome] #','.join(params['ncrna_keys'][genome])
ncrna

### Peak file names

merge = "data/peaksInfo/samples.merge"
raw_peaks="data/peaksInfo/peaks.merge.tss"
minimal="data/peaksInfo/peaks.merge.tss.minimal"


### Peaks info on samples and the average values. Also uses the thresholds for confident peaks here

peaks_sourceMeta = "data/peaksInfo/peaks.sourceMeta.tsv"
peaks_sourceConfident = "data/peaksInfo/peaks.sourceMeta.confident.index"
expr_thresh = 1 #cpm threshold
num_samples_thresh = 1

### Using the annotation

homeranno= f"data/{genome}/anno/peaks.merge.tss.anno"
peaks_minimal = f"data/{genome}/anno/peaks.merge.tss.minimal.anno"
peaks_homeranno = f"data/{genome}/anno/peaks.merge.tss.minimal.anno"
peaks_distance = f"data/{genome}/anno/peaks.distance.tsv"

### Filter introns and cds names

noncds_peaks = f"data/{genome}/genebody_filters/peaks.nocds.index"
nonintron_peaks = f"data/{genome}/genebody_filters/peaks.nointron.index"
non_cds_intron_peaks = f"data/{genome}/genebody_filters/peaks.nocds.nointron.index"

### revised TSS

pc_tss = f"data/{genome}/refType/pc.index"
ncrna_tss = f"data/{genome}/refType/ncrna.index"
intergenic_tss = f"data/{genome}/refType/ncrna.index"


# filtered tss
allTSS_f = f"data/{genome}/allTSS/allTSS_thresh{thresh}_numSamp{numSamp}.tsv"

In [13]:
df = pd.read_csv(allTSS_f, index_col=0)
df

pc = df[df['refType'] == 'mRNA']
pc

refTSS = pd.read_csv(reftss_f, index_col=0, sep='\t')
refTSS

## Get number of genes

pc['Gene'] = pc['Nearest TSS'].map(refTSS['gene'].to_dict())
pc

pc.groupby('Gene').plot.bar()

,ID,Distance to Nearest TSS,Nearest TSS,Distance to Nearest TSS_Rev,Nearest TSS_Rev,Annotation,homeranno,refType,isCDS,isIntron,isAbove_numsamples,isCHO
2,2,60370.0,XM_027432388.1,-746.0,XM_027432478.1,mRNA,promoter-TSS (ID=exon-XM_027391402.1-1;Parent=...,divergent,False,False,True,Overlap
4,4,-11.0,XM_027421990.1,-53290.0,XM_027421992.1,mRNA,promoter-TSS (ID=exon-XM_027422995.1-6;Parent=...,mRNA,False,False,True,Overlap
5,5,296.0,XM_027428527.1,24329.0,XM_027428469.1,mRNA,Intergenic,mRNA,False,False,True,Tissues
6,6,47.0,XM_027418757.1,377336.0,XM_027418744.1,mRNA,promoter-TSS (ID=exon-XR_003480724.1-1;Parent=...,mRNA,False,False,True,Overlap
7,7,118811.0,XM_027424703.1,191.0,XM_027424708.1,mRNA,Intergenic,divergent,False,False,True,Overlap
...,...,...,...,...,...,...,...,...,...,...,...,...
532240,536617,-406001.0,XM_027407647.1,790639.0,XM_027407642.1,mRNA,Intergenic,Intergenic,False,False,True,Tissues
532249,536626,-17151.0,XM_027407847.1,1956.0,XM_027407848.1,mRNA,Intergenic,Intergenic,False,False,True,Tissues
532252,536629,-113749.0,XM_027407818.1,65496.0,XR_003483883.1,mRNA,Intergenic,Intergenic,False,False,True,Tissues
532253,536630,-80520.0,XM_027409383.1,16925.0,XM_027409587.1,mRNA,Intergenic,Intergenic,False,False,True,Overlap


## 1. process genome 

In [4]:
create_anno_tss(reftss_f, gff_f, pc, ncrna, break_char='=')

col_names [0, 1, 2, 3, 4, 5, 6, 7, 8]


54410it [00:14, 3832.00it/s]


,Chr,How,Annotation,Start,End,.1,Strand,.2,ID,actual_start,...,gene,model_evidence,product,transcript_id,Note,exception,inference,end_range,partial,start_range
XR_003479383.1,NW_020822366.1,Gnomon,lnc_RNA,269294,269294,.,+,.,rna-XR_003479383.1,269294,...,LOC113832197,Supporting evidence includes similarity to: 10...,uncharacterized LOC113832197%2C transcript var...,XR_003479383.1,NaN,NaN,NaN,NaN,NaN,NaN
XR_003479384.1,NW_020822366.1,Gnomon,lnc_RNA,551299,551299,.,+,.,rna-XR_003479384.1,551299,...,LOC113832198,Supporting evidence includes similarity to: 10...,uncharacterized LOC113832198,XR_003479384.1,NaN,NaN,NaN,NaN,NaN,NaN
XR_003479386.1,NW_020822366.1,Gnomon,lnc_RNA,696459,696459,.,+,.,rna-XR_003479386.1,696459,...,LOC103159926,Supporting evidence includes similarity to: 10...,uncharacterized LOC103159926%2C transcript var...,XR_003479386.1,NaN,NaN,NaN,NaN,NaN,NaN
XR_003479388.1,NW_020822366.1,Gnomon,lnc_RNA,696459,696459,.,+,.,rna-XR_003479388.1,696459,...,LOC103159926,Supporting evidence includes similarity to: 10...,uncharacterized LOC103159926%2C transcript var...,XR_003479388.1,NaN,NaN,NaN,NaN,NaN,NaN
XR_003479385.1,NW_020822366.1,Gnomon,lnc_RNA,710127,710127,.,+,.,rna-XR_003479385.1,710127,...,LOC103159926,Supporting evidence includes similarity to: 10...,uncharacterized LOC103159926%2C transcript var...,XR_003479385.1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XM_027434593.1,NW_020824186.1,Gnomon,mRNA,29432,29432,.,+,.,rna-XM_027434593.1,29432,...,LOC103158952,Supporting evidence includes similarity to: 2 ...,PRAME family member 12-like%2C transcript vari...,XM_027434593.1,NaN,NaN,NaN,NaN,NaN,NaN
XM_027434595.1,NW_020824188.1,Gnomon,mRNA,2147,2147,.,+,.,rna-XM_027434595.1,2147,...,LOC113839149,Supporting evidence includes similarity to: 25...,zinc finger protein 431-like%2C transcript var...,XM_027434595.1,NaN,NaN,NaN,NaN,NaN,NaN
XM_027434594.1,NW_020824188.1,Gnomon,mRNA,2147,2147,.,+,.,rna-XM_027434594.1,2147,...,LOC113839149,Supporting evidence includes similarity to: 30...,zinc finger protein 431-like%2C transcript var...,XM_027434594.1,NaN,NaN,NaN,NaN,NaN,NaN
XM_027434596.1,NW_020824188.1,Gnomon,mRNA,2147,2147,.,+,.,rna-XM_027434596.1,2147,...,LOC113839149,Supporting evidence includes similarity to: 27...,zinc finger protein 431-like%2C transcript var...,XM_027434596.1,NaN,NaN,NaN,NaN,NaN,NaN


## 2a. Create minimal

In [5]:
print('samples merge', merge)
peaks_df = pd.read_csv(merge, sep="\t", index_col=0)
peaks_df["Start"] = (
        1.0 * (peaks_df["End"] + peaks_df["Start"]) / 2).astype(int)
peaks_df["End"] = peaks_df["Start"]  # length 1
peaks_df["actual_start"] = peaks_df["End"]
peaks_df.head()
peaks_df.to_csv(raw_peaks, sep="\t", index=False)
peaks_df[['Chr', 'Start', 'End', 'Strand', 'Stat']].to_csv(minimal, sep='\t')

samples merge data/peaksInfo/samples.merge


/home/isshamie/software/anaconda2/envs/TSS/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 2b Create homeranno

#### in snakemake

## 3. Combine minimal and anno

In [194]:
anno_df

ID
1-539324    promoter-TSS (ID=exon-XR_003482412.1-1;Parent=...
1-353478    promoter-TSS (ID=exon-XR_003487470.1-1;Parent=...
1-146       promoter-TSS (ID=exon-XM_027391402.1-1;Parent=...
1-349135    promoter-TSS (ID=exon-XR_003483964.1-10;Parent...
1-300006    promoter-TSS (ID=exon-XM_027422995.1-6;Parent=...
                                  ...                        
1-295553                                           Intergenic
1-295682                                           Intergenic
1-305616                                           Intergenic
1-285023                                           Intergenic
1-287900                                           Intergenic
Name: Annotation, Length: 540127, dtype: object

In [197]:
merge_df = pd.read_csv(minimal, index_col=0, sep='\t')
anno_df = pd.read_csv(homeranno, sep='\t', na_filter=False)
anno_df.index.name = "ID"
anno_df = anno_df['Annotation']
merge_df = pd.concat((merge_df, anno_df), axis=1)
print('merge_df')
(merge_df.head())

merge_df


,Chr,Start,End,Strand,Stat,Annotation
ID,,,,,,
0,NW_020822452.1,16590385,16590385,+,16.227333,promoter-TSS (ID=exon-XR_003482412.1-1;Parent=...
1,NW_020822597.1,23595396,23595396,+,5.740000,promoter-TSS (ID=exon-XR_003487470.1-1;Parent=...
2,NW_020822680.1,728718,728718,+,7.579579,promoter-TSS (ID=exon-XM_027391402.1-1;Parent=...
3,NW_020822602.1,10883205,10883205,+,3.992000,promoter-TSS (ID=exon-XR_003483964.1-10;Parent...
4,NW_020822601.1,37427907,37427907,+,129.031738,promoter-TSS (ID=exon-XM_027422995.1-6;Parent=...


In [6]:
merge_df = pd.read_csv(minimal, index_col=0, sep='\t')
anno_df = pd.read_csv(homeranno, sep='\t', na_filter=False, index_col=0)
anno_df.index.name = "ID"
anno_df = anno_df['Annotation']
merge_df = pd.concat((merge_df, anno_df), axis=1)
print('merge_df')
print(merge_df.head())
merge_df.to_csv(peaks_homeranno, sep='\t')

merge_df
               Chr     Start       End Strand        Stat  \
ID                                                          
0   NW_020822452.1  16590385  16590385      +   16.227333   
1   NW_020822597.1  23595396  23595396      +    5.740000   
2   NW_020822680.1    728718    728718      +    7.579579   
3   NW_020822602.1  10883205  10883205      +    3.992000   
4   NW_020822601.1  37427907  37427907      +  129.031738   

                                           Annotation  
ID                                                     
0   promoter-TSS (ID=exon-XM_027399668.1-1;Parent=...  
1                                          Intergenic  
2   promoter-TSS (ID=exon-XM_027432478.1-3;Parent=...  
3                                          Intergenic  
4   promoter-TSS (ID=exon-XM_027421990.1-1;Parent=...  


## 4. Get distance to TSS

In [16]:
def get_distance_anno(peaks_df, in_anno_df, nb_workers=32):
    """ Get nearest distance and if same strand

    :param peaks_df:
    :return:
    """
    
    anno_chr_str = in_anno_df.sort_values(['Chr','Strand']).set_index(['Chr', "Strand"])
    assert((peaks_df['Start']-peaks_df['End']==0).all())

    def get_nearest_info(x, isSame, col_id="transcript_id"):
        # peaks of one gene
        curr_str = x['Strand']
        assert (curr_str == "+" or curr_str == "-")
        if not isSame:
            if curr_str == "-":
                curr_str = "+"
            else:
                curr_str = "-"
            name='TSS_Rev'
        else:
            name="TSS"

        if (x['Chr'], curr_str) not in anno_chr_str.index:
            return pd.Series([np.nan, ""],
                             index=['Distance to Nearest '+name,
                                    f'Nearest ' + name])
        curr_anno = anno_chr_str.loc[(x['Chr'], curr_str)].reset_index()
        dist_to_peak = x['Start']-curr_anno['actual_start']
        
        min_idx = np.abs(dist_to_peak).idxmin()
        min_dist = dist_to_peak.loc[min_idx]
        min_id = curr_anno.loc[min_idx, col_id]
        return pd.Series([min_dist, min_id], index=[f'Distance to Nearest {name}', f'Nearest {name}'])

    nearest_peaksFw = peaks_df.parallel_apply(get_nearest_info, axis=1,
                                              args=(True,))
    nearest_peaksRev = peaks_df.parallel_apply(get_nearest_info, axis=1,
                                               args=(False,))
    peaks_anno_dist = pd.merge(nearest_peaksFw, nearest_peaksRev, left_index=True, right_index=True)
    return peaks_anno_dist


In [20]:
pandarallel.initialize(nb_workers=cpu, verbose=2, progress_bar=True)

New pandarallel memory created - Size: 2000 MB
Pandarallel will run on 32 workers


In [21]:
reftss = pd.read_csv(reftss_f, sep='\t', index_col=0)
reftss['actual_start'] = reftss['actual_start'].astype(int)
peaks_df = pd.read_csv(peaks_homeranno, sep="\t", index_col=0)
#peaks_df['actual_start'] = peaks_df['actual_start'].astype(int)
peaks_anno_dist = get_distance_anno(peaks_df, reftss,
                                    nb_workers=cpu)
peaks_anno_dist.to_csv(peaks_distance,sep='\t')

## 5. Extract and remove CDS and intron peaks

## 5a. get intron bed file - uses R GenomeRanges
## 5b. get CDS bed file (should already have)
## 5c. remove the filters

In [30]:
from tss.config import HOMER_PATH
import os

In [72]:
"""
Function that uses bedtools intersect to not have any peaks that
overlap with Coding sequence since that would be harder to call a start site (even if alternative start sites).
Will make bed file of this and tsv file.

"""
# Create temporary bed file and then convert to peak-like
# Create bed
cmd = f"{HOMER_PATH}/pos2bed.pl {peaks_homeranno} > {peaks_homeranno}.bed"
print(cmd)
os.system(cmd)

#-v says get the non-intersecting peaks
cmd = f"bedtools intersect -s -v -a {peaks_homeranno}.bed -b {cds_f} > {noncds_peaks}.bed"

print(cmd)
os.system(cmd)

bed_df = pd.read_csv(f"{noncds_peaks}.bed", sep='\t', header=None)
bed_df[3].to_csv(noncds_peaks, index=None, header=False)


/data2/resources/software/homer/v4.11_10242019/bin///pos2bed.pl data/GCF/anno/peaks.merge.tss.minimal.anno > data/GCF/anno/peaks.merge.tss.minimal.anno.bed
bedtools intersect -s -v -a data/GCF/anno/peaks.merge.tss.minimal.anno.bed -b data/input/CDS.bed > data/GCF/genebody_filters/peaks.nocds.index.bed


/home/isshamie/software/anaconda2/envs/TSS/lib/python3.7/site-packages/ipykernel_launcher.py:22: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


In [78]:
"""
Function that uses bedtools intersect to not have any peaks that
overlap with Coding sequence since that would be harder to call a start site (even if alternative start sites).
Will make bed file of this and tsv file.

"""

#-v says get the non-intersecting peaks
cmd = f"bedtools intersect -s -v -a {peaks_homeranno}.bed -b {intron_f} > {nonintron_peaks}.bed"

print(cmd)
os.system(cmd)

bed_df = pd.read_csv(f"{nonintron_peaks}.bed", sep='\t', header=None)
bed_df[3].to_csv(nonintron_peaks, index=None, header=False)


bedtools intersect -s -v -a data/GCF/anno/peaks.merge.tss.minimal.anno.bed -b data/GCF/genome/introns.bed > data/GCF/genebody_filters/peaks.nointron.index.bed


## 6. Call pc and ncrna genes by distance and get refType

In [174]:
peaks_distance=f"data/{genome}/anno/peaks.distance.tsv"
refTSS=f"data/{genome}/genome/refTSS.tsv"
homeranno=f"data/{genome}/anno/peaks.merge.tss.anno"

#output
refType = f"data/{genome}/refType/peaks.refType.tsv"





In [179]:
#params
intergenic_dist=1000
bins=(-1000,1000)

In [180]:
df = pd.read_csv(peaks_distance,sep='\t', index_col=0)
ref_df=pd.read_csv(refTSS,sep='\t', index_col=0)
homeranno_df = pd.read_csv(homeranno,sep='\t', index_col=0)

print('df before dropping no nearest tss', df.shape)
df = df.dropna(subset=["Nearest TSS"])
print('df after dropping no nearest tss', df.shape)
df['Annotation'] = df["Nearest TSS"].map(ref_df["Annotation"].to_dict())

df['homeranno'] = homeranno_df.loc[df.index, 'Annotation']
df

df.head()

# get intergenic
intergenic = df[(df['homeranno'].str.upper() == "INTERGENIC") &
                (df['Distance to Nearest TSS']<intergenic_dist)]

intergenic

promoter = df[(df['Distance to Nearest TSS']>bins[0]) & (df['Distance to Nearest TSS']<bins[1])]
promoter

ncrna


pc_promoters = promoter[promoter['Annotation']==pc]
ncrna_promoters = promoter[promoter['Annotation'].isin(ncrna)]
print(pc_promoters.shape)
pc_promoters.head()

print('Total', pc_promoters.shape[0])
print('annotated as promoter by homer', pc_promoters['homeranno'].str.contains("promoter").sum())

print(ncrna_promoters.shape)
ncrna_promoters.head()

nonpromoters = df.drop(promoter.index)
divergent = nonpromoters[(nonpromoters['Distance to Nearest TSS_Rev']>bins[0]) &
            (nonpromoters['Distance to Nearest TSS_Rev']<bins[1])]
print('divergent', divergent.shape)

df['refType'] = None
df.loc[intergenic.index, 'refType'] = "Intergenic"
df.loc[pc_promoters.index, 'refType'] = "mRNA"
df.loc[ncrna_promoters.index, 'refType'] = "ncRNA"
df.loc[divergent.index, 'refType'] = "divergent"
df

df.to_csv(refType,sep='\t')

%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
sns.countplot(df['refType'])
plt.savefig(refType+".png")

df before dropping no nearest tss (540127, 4)
df after dropping no nearest tss (535730, 4)


/home/isshamie/software/anaconda2/envs/TSS/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/isshamie/software/anaconda2/envs/TSS/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,Distance to Nearest TSS,Nearest TSS,Distance to Nearest TSS_Rev,Nearest TSS_Rev,Annotation,homeranno
ID,,,,,,
0,117319.0,XM_027399667.1,462.0,XM_027399668.1,mRNA,promoter-TSS (ID=exon-XM_027399668.1-1;Parent=...
1,-23850.0,XM_027420351.1,162428.0,XM_027420364.1,mRNA,Intergenic
2,60370.0,XM_027432388.1,-746.0,XM_027432478.1,mRNA,promoter-TSS (ID=exon-XM_027432478.1-3;Parent=...
3,9079.0,XM_027423054.1,14247.0,XM_027422736.1,mRNA,Intergenic
4,-11.0,XM_027421990.1,-53290.0,XM_027421992.1,mRNA,promoter-TSS (ID=exon-XM_027421990.1-1;Parent=...
...,...,...,...,...,...,...
540122,54872.0,XM_027415965.1,58.0,XM_027415967.1,mRNA,promoter-TSS (ID=exon-XM_027415967.1-1;Parent=...
540123,-7856.0,XM_027405626.1,149995.0,XR_003483403.1,mRNA,Intergenic
540124,107085.0,XR_003480196.1,-6829.0,XR_003480188.1,lnc_RNA,Intergenic


In [231]:
above_thresh = tissues>1
above_thresh

,MiscOrgans,Liver,Brain,Lung,Heart,CHO,Pancreas,FemaleReproductive,Muscle,Kidney,Spleen,BMDM
0,False,False,False,True,False,True,False,False,False,True,False,True
1,True,True,True,True,False,False,False,True,False,True,True,False
2,False,False,True,True,False,True,True,True,True,True,True,True
3,True,False,True,True,False,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...
540122,False,False,False,False,False,False,True,False,False,False,True,True
540123,False,False,False,True,False,False,False,False,False,False,True,True
540124,False,False,False,True,False,False,False,False,False,False,True,True
540125,True,False,False,False,False,False,False,False,False,False,False,True


In [245]:
a=above_thresh.loc[above_thresh["CHO"]].index
b=above_thresh.loc[above_thresh.drop("CHO", axis=1).any(axis=1)].index
tot=above_thresh.loc[above_thresh.any(axis=1)].index

tissues.loc[set(a).intersection(set(b)), "isCHO"] = "Overlap"
tissues.loc[set(a)-(set(b)), "isCHO"] = "CHO"
tissues.loc[set(b)-set(a), "isCHO"] = "Tissues"
tissues.loc[set(tot)-(set(a).union(set(b))), "isCHO"] = "None"

In [246]:
import tss.utils.sankey_helper as sh

In [267]:
genebody = pd.read_csv(f"data/{genome}/refType/peaks.refType.geneBody.tsv", sep='\t', index_col=0)
genebody

,Distance to Nearest TSS,Nearest TSS,Distance to Nearest TSS_Rev,Nearest TSS_Rev,Annotation,homeranno,refType,isCDS,isIntron
ID,,,,,,,,,
0,117319.0,XM_027399667.1,462.0,XM_027399668.1,mRNA,promoter-TSS (ID=exon-XR_003482412.1-1;Parent=...,divergent,True,True
1,-23850.0,XM_027420351.1,162428.0,XM_027420364.1,mRNA,promoter-TSS (ID=exon-XR_003487470.1-1;Parent=...,NaN,False,True
2,60370.0,XM_027432388.1,-746.0,XM_027432478.1,mRNA,promoter-TSS (ID=exon-XM_027391402.1-1;Parent=...,divergent,False,False
3,9079.0,XM_027423054.1,14247.0,XM_027422736.1,mRNA,promoter-TSS (ID=exon-XR_003483964.1-10;Parent...,NaN,False,False
4,-11.0,XM_027421990.1,-53290.0,XM_027421992.1,mRNA,promoter-TSS (ID=exon-XM_027422995.1-6;Parent=...,mRNA,False,False
...,...,...,...,...,...,...,...,...,...
540122,54872.0,XM_027415965.1,58.0,XM_027415967.1,mRNA,Intergenic,divergent,False,False
540123,-7856.0,XM_027405626.1,149995.0,XR_003483403.1,mRNA,Intergenic,Intergenic,False,False
540124,107085.0,XR_003480196.1,-6829.0,XR_003480188.1,lnc_RNA,Intergenic,NaN,False,False


In [270]:
thresh=1
tissues_score = pd.read_csv(f'data/peaksInfo/tissues_scoring.thresh{thresh}.tsv',sep='\t',index_col=0)
tissues_score

,num_above_cpm,isAbove_numsamples,isCHO
0,4,True,Overlap
1,7,True,Tissues
2,9,True,Overlap
3,10,True,Overlap
4,12,True,Overlap
...,...,...,...
540122,3,True,Tissues
540123,3,True,Tissues
540124,3,True,Tissues
540125,2,False,Tissues
